# Capstone: What Makes a Good Board Game?

* **Author**: Winnie Zhang
* Next Notebook: 2. Feature Engineering

## Introduction
What makes a good boardgame?

In this project, my main goals are to understand the language used when describing boardgames. Particularly, I want to answer the following questions:
1. What words are used by reviewers that positively impact ratings? What words negatively impact ratings?
- By knowing which words that positively impact games, companies know what type of games they hsould focus on

2. Are certain types of board games more popular than others?
- Certain categories? Co-op? Games that include certain mechanisms?

This project will use a data scraped from the website [BoardGameGeek](https://boardgamegeek.com/). The dataset was scraped and uploaded by a user to Kaggle [here](https://www.kaggle.com/datasets/jvanelteren/boardgamegeek-reviews?select=bgg-15m-reviews.csv). The dataset contains 19 million rows of data and is updated as of January 2022. 

The goal of this notebook is to clean and preprocess the data.

## Cleaning and Preprocessing
First, I will import all the packages that I need. 

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import joblib

Next, I will read in the data.

In [34]:
reviews = pd.read_csv("data/bgg-19m-reviews.csv")
games = pd.read_csv("data/games_detailed_info.csv")
game_reviews = pd.read_csv("data/2022-01-08.csv")

C:\Users\winni\AppData\Local\Temp\ipykernel_26508\1759049023.py:2: DtypeWarning: Columns (29,48) have mixed types. Specify dtype option on import or set low_memory=False.
  games = pd.read_csv("data/games_detailed_info.csv")


The dataset is made of 3 different tables: `reviews`, `games`, and `game_reviews`:
- `reviews` contains all the user ratings and comments left for each game
- `game_reviews`  contains description of the game, its average rating, and ranking
- `games` contains other information about the game such as the categories it falls under, the number of comments, the difficulty, and so on.

I will take a look at the first 5 rows of each.

In [35]:
reviews.head()

,Unnamed: 0,user,rating,comment,ID,name
0,0,Torsten,10.0,NaN,30549,Pandemic
1,1,mitnachtKAUBO-I,10.0,Hands down my favorite new game of BGG CON 200...,30549,Pandemic
2,2,avlawn,10.0,I tend to either love or easily tire of co-op ...,30549,Pandemic
3,3,Mike Mayer,10.0,NaN,30549,Pandemic
4,4,Mease19,10.0,This is an amazing co-op game. I play mostly ...,30549,Pandemic


In [36]:
game_reviews.head()

In [37]:
games.head()

`games` and `game_reviews` seem to contain some of the same information, such as `Year`, `Rank`, `Average`, `Bayes average` and `Users rated`. 

Since `games` has much more information than `game_reviews`, including the difficulty (`weight`), `playingtime` and so on, I will continue exploring the `games` dataframe and no longer look at `game_reviews.

## `Reviews` DataFrame
### Inspecting the Data
First, I will take a look at the `reviews` DataFrame. I will inspect each column of the dataset.

In [38]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18964807 entries, 0 to 18964806
Data columns (total 6 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   user        object 
 2   rating      float64
 3   comment     object 
 4   ID          int64  
 5   name        object 
dtypes: float64(1), int64(2), object(3)
memory usage: 868.1+ MB


- `Unnamed: 0`: This column appears to be the same as the index column. I will check to see if they're the same and if they are, drop it.
- `user`: This column is the username of the person who left the review. It may be useful to feature engineer how many times a user leaves a review and the average score they give. After engineering these features, this column can be dropped.
- `rating`: This is the rating of the boardgame that the user gave.
- `comment`: The user also has an option to leave a comment with their review. 
- `ID` is the ID of the boardgame. This can be used later, to concatenate the `reviews` and `games` dataframes.
- `name` is the name of the boardgame. This can be dropped later as it doesn't add valuable information.

Next, I will inspect the same of the `reviews` dataframe.

In [39]:
reviews.shape

(18964807, 6)

This dataset consists of almost 19 million rows and 6 columns.

### Dropping Unneeded Columns
First, I will check to see in the `Unnamed: 0` column is the same as index. If it is, I will drop the column.

In [40]:
(reviews["Unnamed: 0"] != reviews.index).sum()

0

There are no rows where `Unnamed: 0` doesn't match the index of the column. Therefore, I will drop this column. 

In [41]:
reviews.drop(["Unnamed: 0"], axis=1, inplace=True)

# sanity check
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18964807 entries, 0 to 18964806
Data columns (total 5 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user     object 
 1   rating   float64
 2   comment  object 
 3   ID       int64  
 4   name     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 723.5+ MB


I succesfully dropped `Unnamed: 0`. I will leave the `name` column in for now so I can double check that the `reviews` and `games` column are concatenated properly later on.

### Duplicated Rows and Columns
Next, I will check if any of the rows are duplicated.

In [42]:
reviews.duplicated().sum()

0

None of the rows are duplicated so they won't need to be dropped. Based on the first 5 rows of the dataframe, none of the columns appear to be duplicated so I will not check them for duplication.

### Missing Values

Next, I will check if there are missing values in any of these rows.

In [43]:
reviews.isna().sum()

user             66
rating            0
comment    15596188
ID                0
name              0
dtype: int64

There are over 15 million rows misisng in the `comments` column since leaving a comment is optional. Since the whole purpose of my project is to make predictions based on these comments, I will drop all the rows where the comments are missing.

In [44]:
18_964_807 - 15_596_188

3368619

This still leaves us with approximately 3 million rows of data, which is still a lot. 

In [45]:
# take rows where comment column isn't NaN
reviews_clean = reviews[reviews["comment"].notna()]

# sanity check
reviews_clean.head()

,user,rating,comment,ID,name
1,mitnachtKAUBO-I,10.0,Hands down my favorite new game of BGG CON 200...,30549,Pandemic
2,avlawn,10.0,I tend to either love or easily tire of co-op ...,30549,Pandemic
4,Mease19,10.0,This is an amazing co-op game. I play mostly ...,30549,Pandemic
5,cfarrell,10.0,Hey! I can finally rate this game I've been pl...,30549,Pandemic
8,gregd,10.0,Love it- great fun with my son. 2 plays so far...,30549,Pandemic


In [46]:
# check shape of clean dataset
reviews_clean.shape

(3368619, 5)

There are 3.3 million rows left in our dataset!

Next, I will check if there are any missing rows left.

In [47]:
# check if there are any Nan rows left
reviews_clean.isna().sum()

user       0
rating     0
comment    0
ID         0
name       0
dtype: int64

There are no null rows left. Now, I can reset the index.

In [48]:
# reset index 
reviews_clean.reset_index(drop=True, inplace=True)

# sanity check
reviews_clean.head()

,user,rating,comment,ID,name
0,mitnachtKAUBO-I,10.0,Hands down my favorite new game of BGG CON 200...,30549,Pandemic
1,avlawn,10.0,I tend to either love or easily tire of co-op ...,30549,Pandemic
2,Mease19,10.0,This is an amazing co-op game. I play mostly ...,30549,Pandemic
3,cfarrell,10.0,Hey! I can finally rate this game I've been pl...,30549,Pandemic
4,gregd,10.0,Love it- great fun with my son. 2 plays so far...,30549,Pandemic


In [49]:
reviews_clean2 = reviews_clean.copy()

In [50]:
# keep columns that don't start with this 
reviews_clean2 = reviews_clean2[~reviews_clean2["comment"].str.contains("^[0-9]|10 ->",
                                                                        regex=True)]

In [51]:
reviews_clean2.shape

(3230919, 5)

In [52]:
3368619 - 3230919

137700

In [53]:
reviews_clean2["comment"] = reviews_clean2["comment"].str.\
    replace("(10|[0-9]) \-\> (10|[0-9])", "")

C:\Users\winni\AppData\Local\Temp\ipykernel_26508\1122396937.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews_clean2["comment"] = reviews_clean2["comment"].str.\


In [54]:
reviews_clean2["comment"] = reviews_clean2["comment"].str.strip()

In [55]:
# reset index 
reviews_clean2.reset_index(inplace=True)

Since some of the columns might contain a non-English review, I will try to remove all the non-English reviews.

In [56]:
reviews_clean3 = reviews_clean2[reviews_clean2['comment'].map(lambda x: x.isascii())]

In [57]:
# sanity check
reviews_clean3.shape

(3038471, 6)

There are now only 3.1 million columns. 

During this process, I also noticed that some `comments` only contain a rating change and sometimes, this rating change is included at the end of the user's review. I want to drop columns that only contain the rating change and remove the rating change that occurs at the end of comments.

The `reviews_clean` table is now clean! I will now save this dataframe.

**NOTE**: I will be saving all my dataframes as `pkl` files instead of `csv` because when I save them as `csv` files and upload them again later, I've been noticing an issue where random null values appear in my dataset.

In [58]:
joblib.dump(reviews_clean3, "data/reviews_clean_final.pkl")

['data/reviews_clean_final.pkl']

I will now take a look at the `games` dataframe.

## `games` DataFrame

## Inpsecting Games
First, I will take a look at the `games` dataframe.

In [59]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21631 entries, 0 to 21630
Data columns (total 56 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     21631 non-null  int64  
 1   type                           21631 non-null  object 
 2   id                             21631 non-null  int64  
 3   thumbnail                      21616 non-null  object 
 4   image                          21616 non-null  object 
 5   primary                        21631 non-null  object 
 6   alternate                      8850 non-null   object 
 7   description                    21630 non-null  object 
 8   yearpublished                  21631 non-null  int64  
 9   minplayers                     21631 non-null  int64  
 10  maxplayers                     21631 non-null  int64  
 11  suggested_num_players          21631 non-null  object 
 12  suggested_playerage            21520 non-null 

There are a lot of columns and a lot of them appear to have mostly null values. Those columns would likely be dropped. I will inpsect the columns in more detail in the next part of the notebook.

I will now check the shape of the dataframe.

In [60]:
games.shape

(21631, 56)

`games` consists of 21,631 rows and 56 different columns.

### Exploring and Dropping Columns

First, I will check if `Unnamed: 0` is the same as the index for `games`.

In [61]:
(games["Unnamed: 0"] != games.index).sum()

0

`Unnamed: 0` and the `index` appear to be the same so I will drop `Unnamed: 0`.

In [62]:
games.drop("Unnamed: 0", axis=1, inplace=True)

Next, I will inspect the `type` column more closely.

In [63]:
games["type"]

0        boardgame
1        boardgame
2        boardgame
3        boardgame
4        boardgame
           ...    
21626    boardgame
21627    boardgame
21628    boardgame
21629    boardgame
21630    boardgame
Name: type, Length: 21631, dtype: object

This appears to be labelling the game as if it's a boardgame or not. I will take a look at all the unique values in `type`.

In [64]:
games["type"].value_counts()

boardgame    21631
Name: type, dtype: int64

There is only 1 type: boardgame. Therefore, this column isn't very useful and I'll drop it.

`id` is the next column. This is the board game id and will be useful for concatenating the `reviews_clean` dataframe and the 

In [65]:
games.drop("type", axis=1, inplace=True)

The next few columns are:
- `Thumbnail`: This is a link to a thumbail. 
- `Image`: This is a link to the image
- `primary`: The name of the board game
- `alternate`: Altnertive names of the game in different languages. This is a list.

I will drop `Thumbnail`, `Image`, and `alternate` as they don't provide extra information. I will also later drop `primary` after I concatenate `games` with `reviews_clean` and confirm all the game names are the same.

In [66]:
games.drop(["thumbnail", "image", "alternate"], axis=1, inplace=True)

Here are the next few columns:
- `description`: The description of the board game.
- `yearpublished`: The year the boardgame was releasde.
- `minplayers`: The minimum number of players required to play the game.
- `maxplayers`: The maximum number of players allowed to play the game.

I will be keeping all of this information as they can contribute to how the board game is rated.

The next few columns I will be looking at are:
- `suggested_num_players`: The number of players suggested users think should play the game. Players vote on the number of players they think should play the game. This is an ordered dictionary containing information on the number of players, and votes on whether or not people recommend this number. 
- `suggested_playerage`: The age users vote on and think should play the game. This is an ordered dictionary containing on the suggested age and the number of votes people of people's suggested age.
- `suggested_language_dependence`: This is the level of language needed to play the game. This is an ordered dictionary containing the level of language needed and the number of votes.

In [67]:
games["suggested_num_players"]

0        [OrderedDict([('@numplayers', '1'), ('result',...
1        [OrderedDict([('@numplayers', '1'), ('result',...
2        [OrderedDict([('@numplayers', '1'), ('result',...
3        [OrderedDict([('@numplayers', '1'), ('result',...
4        [OrderedDict([('@numplayers', '1'), ('result',...
                               ...                        
21626    [OrderedDict([('@numplayers', '1'), ('result',...
21627    [OrderedDict([('@numplayers', '1'), ('result',...
21628    [OrderedDict([('@numplayers', '1'), ('result',...
21629    [OrderedDict([('@numplayers', '1'), ('result',...
21630    [OrderedDict([('@numplayers', '1'), ('result',...
Name: suggested_num_players, Length: 21631, dtype: object

These columns are difficult to deal clean and contain a lot of information so I'll be dropping these columns.

In [68]:
games.drop(["suggested_num_players", "suggested_playerage", "suggested_language_dependence"], 
           axis=1, inplace=True)

Next, I will take a look at the following columns:
- `playingtime`: The max amount of suggested play time.
- `minplaytime`: The min amount of suggested play time.
- `maxplaytime`: The max amount of suggested play time.
- `minage`: The suggested minimum age for the board game.

In [69]:
games[["playingtime", "minplaytime", "maxplaytime", "minage"]]

,playingtime,minplaytime,maxplaytime,minage
0,45,45,45,8
1,45,30,45,7
2,120,60,120,10
3,30,30,30,10
4,30,30,30,13
...,...,...,...,...
21626,30,20,30,8
21627,30,15,30,10
21628,60,60,60,10
21629,10,10,10,5


`playingtime` and `maxplaytime` looks the same. I will check if they're the same.

In [70]:
(games["playingtime"] == games["maxplaytime"]).sum()

21631

21,631 rows are the same. That's the same number of rows as the entire dataset, meaning the `playingtime` and `maxplaytime` are the same. Therefore, I will drop `playingtime`.

In [71]:
games.drop("playingtime", axis=1, inplace=True)

Next, I will look ath the following columns:
- `boardgamecategory`: The categories the board game falls under. This is a list. 
- `boardgamemechanic`: The mechanics involved in the gboard game. This is a list.
- `boardgamefamily`: The family the board game falls under. This is a list.

In [72]:
games[["boardgamecategory", "boardgamemechanic", "boardgamefamily"]].head()

,boardgamecategory,boardgamemechanic,boardgamefamily
0,['Medical'],"['Action Points', 'Cooperative Game', 'Hand Ma...","['Components: Map (Global Scale)', 'Components..."
1,"['City Building', 'Medieval', 'Territory Build...","['Area Majority / Influence', 'Map Addition', ...","['Cities: Carcassonne (France)', 'Components: ..."
2,"['Economic', 'Negotiation']","['Dice Rolling', 'Hexagon Grid', 'Income', 'Mo...","['Animals: Sheep', 'Components: Hexagonal Tile..."
3,"['Ancient', 'Card Game', 'City Building', 'Civ...","['Drafting', 'Hand Management', 'Set Collectio...","['Ancient: Babylon', 'Ancient: Egypt', 'Ancien..."
4,"['Card Game', 'Medieval']","['Deck, Bag, and Pool Building', 'Delayed Purc...","['Crowdfunding: Wspieram', 'Game: Dominion', '..."


I want to split these columns and see which categories, mechanics, and families exist and the number of counts for each.

Since a lot of these columns are null, I will only look at the non-null columns.

In [73]:
# df of just boardgamecategory
category_df = games[games["boardgamecategory"].notnull()]
category_df["boardgamecategory"].head()

0                                          ['Medical']
1    ['City Building', 'Medieval', 'Territory Build...
2                          ['Economic', 'Negotiation']
3    ['Ancient', 'Card Game', 'City Building', 'Civ...
4                            ['Card Game', 'Medieval']
Name: boardgamecategory, dtype: object

Next, I will strip the [] brackets.

In [74]:
category_df["boardgamecategory"] = category_df["boardgamecategory"].\
    map(lambda x: x.lstrip("[").rstrip("]"))

C:\Users\winni\AppData\Local\Temp\ipykernel_26508\41042790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_df["boardgamecategory"] = category_df["boardgamecategory"].\


In [77]:
games["boardgamecategory"] =  category_df["boardgamecategory"]

In [78]:
# sanity check
games["boardgamecategory"].head()

0                                            'Medical'
1    'City Building', 'Medieval', 'Territory Building'
2                            'Economic', 'Negotiation'
3    'Ancient', 'Card Game', 'City Building', 'Civi...
4                              'Card Game', 'Medieval'
Name: boardgamecategory, dtype: object

I will also count the unique categories and see how many fall under each.

Now I will look at `boardgamemechanic`.

In [79]:
mechanic_df = games[games["boardgamemechanic"].notnull()]
mechanic_df["boardgamemechanic"] = mechanic_df["boardgamemechanic"].map(lambda x: x.lstrip("[").\
                                                                        rstrip("]")) 

games["boardgamemechanic"] =  mechanic_df["boardgamemechanic"]

C:\Users\winni\AppData\Local\Temp\ipykernel_26508\3914962758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mechanic_df["boardgamemechanic"] = mechanic_df["boardgamemechanic"].map(lambda x: x.lstrip("[").\


In [80]:
games["boardgamemechanic"].head()

0    'Action Points', 'Cooperative Game', 'Hand Man...
1    'Area Majority / Influence', 'Map Addition', '...
2    'Dice Rolling', 'Hexagon Grid', 'Income', 'Mod...
3    'Drafting', 'Hand Management', 'Set Collection...
4    'Deck, Bag, and Pool Building', 'Delayed Purch...
Name: boardgamemechanic, dtype: object

Now, I will look at `boardgamefamily`. 

In [81]:
family_df = games[games["boardgamefamily"].notnull()] 
family_df["boardgamefamily"] = family_df["boardgamefamily"].map(lambda x: x.lstrip("[").\
                                                                rstrip("]"))
games["boardgamefamily"] =  family_df["boardgamefamily"]

C:\Users\winni\AppData\Local\Temp\ipykernel_26508\1913580934.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  family_df["boardgamefamily"] = family_df["boardgamefamily"].map(lambda x: x.lstrip("[").\


In [82]:
# sanity check
games["boardgamefamily"].head()

0    'Components: Map (Global Scale)', 'Components:...
1    'Cities: Carcassonne (France)', 'Components: M...
2    'Animals: Sheep', 'Components: Hexagonal Tiles...
3    'Ancient: Babylon', 'Ancient: Egypt', 'Ancient...
4    'Crowdfunding: Wspieram', 'Game: Dominion', 'M...
Name: boardgamefamily, dtype: object

Now, I will look at these 2 columns:
- `boardgameexpansion`: an expansion of the board game
- `boardgameimplementation`: Different implementations of the board game (different versions, different rules)

In [83]:
games[["boardgameexpansion", "boardgameimplementation"]].head()

,boardgameexpansion,boardgameimplementation
0,['Pandemic: Gen Con 2016 Promos – Z-Force Team...,"['Pandemic Legacy: Season 0', 'Pandemic Legacy..."
1,"['20 Jahre Darmstadt Spielt', 'Apothecaries (f...","['The Ark of the Covenant', 'Carcassonne für 2..."
2,"['20 Jahre Darmstadt Spielt', 'Brettspiel Adve...","['Baden-Württemberg Catan', 'Catan Geographies..."
3,"['7 Wonders: Armada', '7 Wonders: Babel', '7 W...","['7 Wonders (Second Edition)', '7 Wonders Duel..."
4,"['Ancient Times (fan expansion for Dominion)',...","['Dominion (Second Edition)', 'Het Koninkrijk ..."


In [84]:
games.isnull().sum()

id                             0
primary                        0
description                    1
yearpublished                  0
minplayers                     0
maxplayers                     0
minplaytime                    0
maxplaytime                    0
minage                         0
boardgamecategory            283
boardgamemechanic           1590
boardgamefamily             3761
boardgameexpansion         16125
boardgameimplementation    16769
boardgamedesigner            596
boardgameartist             5907
boardgamepublisher             1
usersrated                     0
average                        0
bayesaverage                   0
Board Game Rank                0
Strategy Game Rank         19294
Family Game Rank           19304
stddev                         0
median                         0
owned                          0
trading                        0
wanting                        0
wishing                        0
numcomments                    0
numweights

A lot of games seem to have null values for these 2 categories. Instead of dropping them, I will encode them as whether or not they have a implementation or expansion (1 if the do, 0 if they don't).

In [85]:
games["boardgameexpansion"] = np.where(games["boardgameexpansion"].isna(), 0, 1)

I will also do this for `boardgameimplementation`.

In [86]:
games["boardgameimplementation"] = np.where(games["boardgameimplementation"].isna(), 0, 1)

Next, I will look at these categories:
- `boardgamedesigner`: The designers who worked on the board game.
- `boardgameartist`: the artists who worked on the board game.
- `boardgamepublisher`: The publishers who pubished the board game in different countries.

In [87]:
games[["boardgamedesigner", "boardgameartist", "boardgamepublisher"]].head()

,boardgamedesigner,boardgameartist,boardgamepublisher
0,['Matt Leacock'],"['Josh Cappel', 'Christian Hanisch', 'Régis Mo...","['Z-Man Games', 'Albi', 'Asmodee', 'Asmodee It..."
1,['Klaus-Jürgen Wrede'],"['Doris Matthäus', 'Anne Pätzke', 'Chris Quill...","['Hans im Glück', '999 Games', 'Albi', 'Bard C..."
2,['Klaus Teuber'],"['Volkan Baga', 'Tanja Donner', 'Pete Fenlon',...","['KOSMOS', '999 Games', 'Albi', 'Asmodee', 'As..."
3,['Antoine Bauza'],"['Dimitri Chappuis', 'Miguel Coimbra', 'Etienn...","['Repos Production', 'ADC Blackfire Entertainm..."
4,['Donald X. Vaccarino'],"['Matthias Catrein', 'Julien Delval', 'Tomasz ...","['Rio Grande Games', '999 Games', 'Albi', 'Bar..."


First, I will look at `boardgamedesigner`:

In [88]:
designer_df = games[games["boardgamedesigner"].notnull()]
designer_df["boardgamedesigner"] = designer_df["boardgamedesigner"].map(lambda x: x.\
                                                                        lstrip("[").rstrip("]"))
games["boardgamedesigner"]=designer_df["boardgamedesigner"]
games["boardgamedesigner"]

C:\Users\winni\AppData\Local\Temp\ipykernel_26508\1661595047.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  designer_df["boardgamedesigner"] = designer_df["boardgamedesigner"].map(lambda x: x.\


0                         'Matt Leacock'
1                   'Klaus-Jürgen Wrede'
2                         'Klaus Teuber'
3                        'Antoine Bauza'
4                  'Donald X. Vaccarino'
                      ...               
21626      'Julia Koerwer', 'Jono Naito'
21627    'Masakazu Takizawa (たきざわ まさかず)'
21628                     '(Uncredited)'
21629                     '(Uncredited)'
21630                   'Eric R. Harvey'
Name: boardgamedesigner, Length: 21631, dtype: object

Now, I will look at `boardgameartist`.

In [89]:
artist_df = games[games["boardgameartist"].notnull()]
artist_df["boardgameartist"] = artist_df["boardgameartist"].map(lambda x: x.\
                                                                lstrip("[").rstrip("]"))
games["boardgameartist"] = artist_df["boardgameartist"] 
games["boardgameartist"].head()

C:\Users\winni\AppData\Local\Temp\ipykernel_26508\3190912686.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artist_df["boardgameartist"] = artist_df["boardgameartist"].map(lambda x: x.\


0    'Josh Cappel', 'Christian Hanisch', 'Régis Mou...
1    'Doris Matthäus', 'Anne Pätzke', 'Chris Quilli...
2    'Volkan Baga', 'Tanja Donner', 'Pete Fenlon', ...
3    'Dimitri Chappuis', 'Miguel Coimbra', 'Etienne...
4    'Matthias Catrein', 'Julien Delval', 'Tomasz J...
Name: boardgameartist, dtype: object

In [90]:
publisher_df = games[games["boardgamepublisher"].notnull()]
publisher_df["boardgamepublisher"] = publisher_df["boardgamepublisher"].map(lambda x: x.\
                                                                            lstrip("[").rstrip("]"))
games["boardgamepublisher"] = publisher_df["boardgamepublisher"]

C:\Users\winni\AppData\Local\Temp\ipykernel_26508\2678130644.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  publisher_df["boardgamepublisher"] = publisher_df["boardgamepublisher"].map(lambda x: x.\


In [91]:
games["boardgamepublisher"].head()

0    'Z-Man Games', 'Albi', 'Asmodee', 'Asmodee Ita...
1    'Hans im Glück', '999 Games', 'Albi', 'Bard Ce...
2    'KOSMOS', '999 Games', 'Albi', 'Asmodee', 'Ast...
3    'Repos Production', 'ADC Blackfire Entertainme...
4    'Rio Grande Games', '999 Games', 'Albi', 'Bard...
Name: boardgamepublisher, dtype: object

Next, I will look at the following columns:
- `usersrated`: The number of users who rated the game.
- `average`: The average rating of the game.
- `bayesaverage`: The average rating of the game, but 100 ratings rated at 5.5 are added to help prevent games with very few reviews being rated very high. If there are a lot of ratings, this wouldn't affect the rating that much. If there are only a few ratings, this would make the rating lower.
- `Board Game Rank`: The overall ranking of the boardgame.

I will be dropping `bayesaverage` as this isn't information might be related to `average` and might not be a reflecting the comments and ratings someone gives.

In [92]:
games.drop("bayesaverage", axis=1, inplace=True)

Next, I will drop all the columns that include `Rank` as they are many null values. I will only keep `Board Game Rank`. I will also drop `std` and `median` as this relates to the `average` column.

In [93]:
column_list = games.columns.to_list()

In [94]:
rank_list = []
for col in column_list:
    if "Rank" in col and col != "Board Game Rank":
        rank_list.append(col)

In [95]:
games.drop(rank_list, axis=1, inplace=True)

In [96]:
games.drop(["stddev", "median"], axis=1, inplace=True)

I will also drop `boardgameintegration` and `boardgamecompilation` as they contain mostly null values.

In [97]:
games.drop(["boardgamecompilation", "boardgameintegration"], axis=1, inplace=True)

In [98]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21631 entries, 0 to 21630
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       21631 non-null  int64  
 1   primary                  21631 non-null  object 
 2   description              21630 non-null  object 
 3   yearpublished            21631 non-null  int64  
 4   minplayers               21631 non-null  int64  
 5   maxplayers               21631 non-null  int64  
 6   minplaytime              21631 non-null  int64  
 7   maxplaytime              21631 non-null  int64  
 8   minage                   21631 non-null  int64  
 9   boardgamecategory        21348 non-null  object 
 10  boardgamemechanic        20041 non-null  object 
 11  boardgamefamily          17870 non-null  object 
 12  boardgameexpansion       21631 non-null  int32  
 13  boardgameimplementation  21631 non-null  int32  
 14  boardgamedesigner     

There are 21,631 rows and 26 different columns.

Games is now clean and I will save it.

In [99]:
games.to_csv("data/games_clean.csv", index=False)

## Data Wrangling
### Games Dataframe

I will wrangle the columns boardgamecategory, boardgamemechanic, boardgamefamily, boardgameexpansion, boardgameimplementation, boardgamedesigner, boardgameartist, and boardgamepublisher as they consist lists. I should make them into separate columns.

First, I will start with `boardgamecategory`.

In [100]:
games_clean = games.copy()

In [101]:
unique_categories = category_df["boardgamecategory"].str.split(",", expand=True).stack().\
    unique().tolist()
unique_categories_count = category_df["boardgamecategory"].str.split(", ", expand=True).\
    stack().value_counts()

# view top 20
unique_categories_count.head(20)

'Card Game'                    6402
'Wargame'                      3820
'Fantasy'                      2681
'Party Game'                   1968
'Dice'                         1847
'Science Fiction'              1666
'Fighting'                     1658
"Children's Game"              1635
'Abstract Strategy'            1545
'Economic'                     1503
'Animals'                      1354
'World War II'                 1212
'Bluffing'                     1206
'Humor'                        1204
'Adventure'                    1181
'Deduction'                    1150
'Miniatures'                   1091
'Action / Dexterity'           1090
'Movies / TV / Radio theme'    1069
'Medieval'                     1013
dtype: int64

There is a high count for the top 20 categories and they all seem to be important. Therefore, I will keep all of them and make them into binary columns in the dataframe.

In [102]:
# list of categories 
cat_list = ['Card Game', 'Wargame', 'Fantasy', 'Party Game', 'Dice', 'Science Fiction', 'Fighting',
            "Children's Game", 'Abstract Strategy', 'Economic', 'Animals', 'World War II',
            'Bluffing', 'Humor', 'Adventure', 'Deduction', 'Miniatures', 'Action / Dexterity',
            'Movies / TV / Radio theme', 'Medieval']

In [103]:
# replace null values with "nocat"
games_clean["boardgamecategory"] = np.where(games_clean["boardgamecategory"].isna(),
                                            "nocat", games_clean["boardgamecategory"])

In [104]:
# loop through categories and make it 1 if the game has that category and 0 if it doesn't 
for cat in cat_list:
    games_clean[f"{cat}"] = games_clean["boardgamecategory"].apply(lambda x: 1 if (f"{cat}" in x) else 0)

In [105]:
# sanity check 
games_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21631 entries, 0 to 21630
Data columns (total 47 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         21631 non-null  int64  
 1   primary                    21631 non-null  object 
 2   description                21630 non-null  object 
 3   yearpublished              21631 non-null  int64  
 4   minplayers                 21631 non-null  int64  
 5   maxplayers                 21631 non-null  int64  
 6   minplaytime                21631 non-null  int64  
 7   maxplaytime                21631 non-null  int64  
 8   minage                     21631 non-null  int64  
 9   boardgamecategory          21631 non-null  object 
 10  boardgamemechanic          20041 non-null  object 
 11  boardgamefamily            17870 non-null  object 
 12  boardgameexpansion         21631 non-null  int32  
 13  boardgameimplementation    21631 non-null  int

Now, I will look at `boardgamefamily`.

In [106]:
unique_family = family_df["boardgamefamily"].str.split(",", expand=True).stack().unique().tolist()
unique_family_count = family_df["boardgamefamily"].str.split(", ", expand=True).stack().\
    value_counts()
unique_family_count.head(20)

'Players: Two Player Only Games'               3619
'Crowdfunding: Kickstarter'                    3146
'Players: Games with Solitaire Rules'          1261
'Admin: Better Description Needed!'            1020
'Components: Miniatures'                        475
'Digital Implementations: Board Game Arena'     390
'Misc: LongPack Games'                          374
'Country: USA'                                  366
'Category: Combinatorial'                       353
'Players: Solitaire Only Games'                 327
'Players: Wargames with Solitaire Rules'        300
'Components: 3-Dimensional (3D)'                296
'Country: France'                               295
'Theme: Food / Cooking'                         255
'Magazine: Strategy & Tactics'                  246
'Ancient: Rome'                                 241
'Country: Germany'                              225
'Country: Japan'                                222
'Components: Dice with Icons'                   216
'Creatures: 

Only 2 categories seem of note:
- Players: 2 player only games
- Crowdfunding: kickstarter

Therefore, I will binarize these 2 families and concatenate them to the dataframe.

In [107]:
family_list = ['Players: Two Player Only Games', 'Crowdfunding: Kickstarter']
games_clean["boardgamefamily"] = np.where(games_clean["boardgamefamily"].isna(),
                                        "nocat", games_clean["boardgamefamily"])


In [108]:
for family in family_list:
    games_clean[f"{family}"] = games_clean["boardgamefamily"].\
    apply(lambda x: 1 if (f"{family}" in x) else 0)

In [109]:
# sanity check
games_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21631 entries, 0 to 21630
Data columns (total 49 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              21631 non-null  int64  
 1   primary                         21631 non-null  object 
 2   description                     21630 non-null  object 
 3   yearpublished                   21631 non-null  int64  
 4   minplayers                      21631 non-null  int64  
 5   maxplayers                      21631 non-null  int64  
 6   minplaytime                     21631 non-null  int64  
 7   maxplaytime                     21631 non-null  int64  
 8   minage                          21631 non-null  int64  
 9   boardgamecategory               21631 non-null  object 
 10  boardgamemechanic               20041 non-null  object 
 11  boardgamefamily                 21631 non-null  object 
 12  boardgameexpansion              

Now, I will look at `boardgamepublisher`:

In [110]:
unique_publisher_count = publisher_df["boardgamepublisher"].str.split(", ", expand=True).stack().value_counts()
unique_publisher_count.head(20)

Ltd.'                    970
Inc.'                    854
'Hasbro'                 597
'(Self-Published)'       594
'Asmodee'                571
'Ravensburger'           520
'Parker Brothers'        486
'(Web published)'        485
'Pegasus Spiele'         480
'999 Games'              477
'Korea Boardgames Co.    474
'KOSMOS'                 429
'Hobby World'            422
'Edge Entertainment'     405
'Milton Bradley'         401
'Rio Grande Games'       395
'Devir'                  384
'Rebel Sp. z o.o.'       374
LLC'                     367
'Hobby Japan'            361
dtype: int64

There are a lot of different publishers and none of the values are particularly high. Furthermore, every board game has several categories and has different publishers for different countries. Therefore, I will drop the `publisher` column.

In [111]:
games_clean.drop("boardgamepublisher", axis=1, inplace=True)

In [112]:
# sanity check
games_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21631 entries, 0 to 21630
Data columns (total 48 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              21631 non-null  int64  
 1   primary                         21631 non-null  object 
 2   description                     21630 non-null  object 
 3   yearpublished                   21631 non-null  int64  
 4   minplayers                      21631 non-null  int64  
 5   maxplayers                      21631 non-null  int64  
 6   minplaytime                     21631 non-null  int64  
 7   maxplaytime                     21631 non-null  int64  
 8   minage                          21631 non-null  int64  
 9   boardgamecategory               21631 non-null  object 
 10  boardgamemechanic               20041 non-null  object 
 11  boardgamefamily                 21631 non-null  object 
 12  boardgameexpansion              

Now, I will look at `boardgameartist`. 

In [113]:
unique_artist_count = artist_df["boardgameartist"].str.split(", ", expand=True).\
    stack().value_counts()
unique_artist_count.head(20)

'Rodger B. MacGowan'       375
'(Uncredited)'             365
'Franz Vohwinkel'          309
'Redmond A. Simonsen'      233
'Mark Simonitch'           217
'Michael Menzel'           171
'Joe Youst'                157
'Charles Kibler'           141
'Oliver Freudenreich'      137
'Klemens Franz'            132
'Claus Stephan'            128
'Harald Lieske'            124
'Nicolás Eskubi'           112
'Dennis Lohausen'          111
'John Kovalic'             101
Jr.'                        79
'Néstor Romeral Andrés'     79
'Christophe Camilotte'      76
'Larry Hoffman'             75
'Vincent Dutrait'           71
dtype: int64

The value count for the artists aren't particularly high so I will drop this column and not binarize any of them.

In [114]:
games_clean.drop("boardgameartist", axis=1, inplace=True)

Now, I will look at `boardgamedesigner`.

In [115]:
unique_designer_count = designer_df["boardgamedesigner"].str.split(", ", expand=True).stack().value_counts()
unique_designer_count.head(20)

'(Uncredited)'         1445
'Reiner Knizia'         329
'Joseph Miranda'        134
'Wolfgang Kramer'       133
'Richard H. Berg'       111
'Bruno Cathala'          95
'Martin Wallace'         92
'Jim Dunnigan'           92
'James Ernest'           91
'Dean Essig'             86
Jr.'                     85
'Michael Schacht'        84
'Frank Chadwick'         81
'Andrew Looney'          74
'Alan R. Moon'           73
'Bruno Faidutti'         73
'Ty Bomba'               70
'Eric M. Lang'           70
'Steve Jackson (I)'      70
'Reinhard Staupe'        69
dtype: int64

Most boardgame designers are uncredited and most of the other counts aren't that high either so I will also drop this column.

In [116]:
games_clean.drop("boardgamedesigner", axis=1, inplace=True)

Since I added the columns for `boardgamecategory`, `boardgamemechanic`, and `boardgamefamily`, I can drop the original columns.

In [117]:
games_clean.drop(["boardgamecategory", "boardgamemechanic", "boardgamefamily"], axis=1, inplace=True)

In [118]:
# sanity check 
games_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21631 entries, 0 to 21630
Data columns (total 43 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              21631 non-null  int64  
 1   primary                         21631 non-null  object 
 2   description                     21630 non-null  object 
 3   yearpublished                   21631 non-null  int64  
 4   minplayers                      21631 non-null  int64  
 5   maxplayers                      21631 non-null  int64  
 6   minplaytime                     21631 non-null  int64  
 7   maxplaytime                     21631 non-null  int64  
 8   minage                          21631 non-null  int64  
 9   boardgameexpansion              21631 non-null  int32  
 10  boardgameimplementation         21631 non-null  int32  
 11  usersrated                      21631 non-null  int64  
 12  average                         

Now, I will save this dataframe.

In [120]:
joblib.dump(games_clean, "data/games_wrangled.pkl")

['data/games_wrangled.pkl']

## Conclusion
In this notebook, we cleaned the notebook by dropping rows and columns. We also binarized some of the columns.

**Next Notebook**: 2. Feature Engineering
- We engineer new features to add onto the dataframes and concatenate the 2 dataframes together